In [131]:
import numpy as np
import pandas as pd

In [132]:
#***********************************************************
#* first step to drop the 21 duplicates that are problematic in the merge (last merge before 
#* saving "data_simple"
#***********************************************************

In [133]:
dfdn_cln = pd.read_table("csv/Dfdn-cln.csv", sep = '^', dtype='object', index_col=False)
dfdn_cln=dfdn_cln.sort_values(['BOFI_NBR','ADDR_1'])
dfdn_nodup=dfdn_cln.drop_duplicates('BOFI_NBR')
print('%s rows deleted' %(dfdn_cln.shape[0]-dfdn_nodup.shape[0]))
#NOTE: ONLY 20 OBS DROPPED VS 21 IN STATA FILE

20 rows deleted


In [134]:
#*********************************************************
#* bring sentences in, add plea barganing (?):
#*********************************************************
#merge dsum-cln with sent-cln on bofi_nbr dfdn_seq_nbr sys_nbr
#delete if not in dsum-cln but in sent-cln
'''
Note: IDs
- bofi_nbr: defendant id
- sys_nbr: case id , it is the same for multiple offenders crime and for different charges
- case_nbr: it is essentially similar to sys_nbr except in 17 cases (8 times because case_nbr is missing)
- chrg_seq_nbr: suppose to identify the charges. Frequently there is only the nb 2 or 3 for eg. Probably because the others are dismissed...
- dfdt_seq_nbr: identify the defendant among a case with several defendant.

- disp_seq_nbr: ? (1 all the time?)
- phonic_key: ?
'''

'\nNote: IDs\n- bofi_nbr: defendant id\n- sys_nbr: case id , it is the same for multiple offenders crime and for different charges\n- case_nbr: it is essentially similar to sys_nbr except in 17 cases (8 times because case_nbr is missing)\n- chrg_seq_nbr: suppose to identify the charges. Frequently there is only the nb 2 or 3 for eg. Probably because the others are dismissed...\n- dfdt_seq_nbr: identify the defendant among a case with several defendant.\n\n- disp_seq_nbr: ? (1 all the time?)\n- phonic_key: ?\n'

In [135]:
dsum_cln = pd.read_table("csv/Dsum-cln.csv", sep = '^', dtype='object', index_col=False)
sent_cln = pd.read_table("csv/Sent-cln.csv", sep = '^', dtype='object', index_col=False)
data_merged = pd.merge(dsum_cln, sent_cln, \
                  on=['BOFI_NBR','DFDN_SEQ_NBR','SYS_NBR'], \
                  how='left')  
print('%s rows' %(data_merged.shape[0]))

312608 rows


In [136]:
#*********************************************************
#* bring arrest date and crime category
#*********************************************************
#merge in [arrest_date charge_type charge_cat charge_class] from areg-cln
#delete if not in data_merged but in areg-cln
'''
label var charge_type "AR=at arrest, IF=by information, IN-by indictment"
label var charge_cat 
label var charge_class "severity of the charge from 1 sever to 8 lenient"
'''

'\nlabel var charge_type "AR=at arrest, IF=by information, IN-by indictment"\nlabel var charge_cat \nlabel var charge_class "severity of the charge from 1 sever to 8 lenient"\n'

In [137]:
areg_clean = pd.read_table("csv/Areg-cln.csv", sep = '^', dtype='object', index_col=False)

In [138]:
data_merged = pd.merge(data_merged, \
                 areg_clean.ix[:,['ARREST_DATE','CHARGE_TYPE','CHARGE_CAT','CHARGE_CLASS',\
                                 'BOFI_NBR','DFDN_SEQ_NBR','SYS_NBR']],\
                 on=['BOFI_NBR','DFDN_SEQ_NBR','SYS_NBR'], \
                 how='left')
print('%s rows' %(data_merged.shape[0]))

312608 rows


In [139]:
#*********************************************************
#* bring judges characteristics
#*********************************************************
#merge on [bofi_nbr dfdn_seq_nbr disp_seq_nbr sent_seq_nbr sys_nbr chrg_seq_nbr ]
#keep all rows

'''CANNOT FIND THIS DATASET'''

'CANNOT FIND THIS DATASET'

In [140]:
#*********************************************************
#* bring SCREANER ada characteristics
#*********************************************************
#merge in ada_cln on ada_code
#delete if not in data_merged

In [141]:
ada_cln = pd.read_table("csv/Ada-cln.csv", sep = '^', \
                           dtype='object', index_col=False)
data_merged = pd.merge(data_merged, \
                 ada_cln,\
                 on='ADA_CODE', \
                 how='left')
print('%s rows' %(data_merged.shape[0]))

312608 rows


In [142]:
#change the name of the variable on ada to clarify that they are info on screaner ada
old_names = ['ADA_CODE', 'ADA_NAME', 'BAR_ADMISSION', 'DOB', 'RACE', 'SEX', 'PARTY', \
             'ALT_PARTY', 'LAW_SCHOOL', 'UNDERGRAD', 'UNVERIFIED_FLAG', 'INACTIVITY_DATE', \
             'DECEASED_FLAG', 'DISCIPLINARY_ACTION_FLAG', 'INEL_DUES_OR_RULES_FLAG', \
             'ADA_NOTES','TRIAL_ADA_CODE']

new_names = ['SADA_CODE', 'SADA_NAME', 'SADA_BAR_ADMISSION', 'SADA_DOB', 'SADA_RACE', \
             'SADA_SEX', 'SADA_PARTY', 'SADA_ALT_PARTY', 'SADA_LAW_SCHOOL', 'SADA_UNDERGRAD', \
             'SADA_UNVERIFIED_FLAG', 'SADA_INACTIVITY_DATE', 'SADA_DECEASED_FLAG', \
             'SADA_DISCIPLINARY_ACTION_FLAG', 'SADA_INEL_DUES_OR_RULES_FLAG', 'SADA_NOTES',\
             'ADA_CODE']

name_dict={}

for i in range(0,len(old_names)):
    name_dict[old_names[i]]=new_names[i]

data_merged = data_merged.rename(columns=name_dict)

In [143]:
#*********************************************************
#* bring TRIAL ada characteristics
#*********************************************************
#merge in ada_cln on ada_code
#delete if not in data_merged

'''TRIAL_ADA_CODE was renamed to ADA_CODE.  We are now merging on TRIAL_ADA_CODE'''

'TRIAL_ADA_CODE was renamed to ADA_CODE.  We are now merging on TRIAL_ADA_CODE'

In [144]:
ada_cln = pd.read_table("csv/Ada-cln.csv", sep = '^', \
                           dtype='object', index_col=False)
data_merged = pd.merge(data_merged, \
                 ada_cln,\
                 on='ADA_CODE', \
                 how='left')

print('%s rows' %(data_merged.shape[0]))

old_names = ['ADA_CODE', 'ADA_NAME', 'BAR_ADMISSION', 'DOB', 'RACE', 'SEX', 'PARTY', \
             'ALT_PARTY', 'LAW_SCHOOL', 'UNDERGRAD', 'UNVERIFIED_FLAG', 'INACTIVITY_DATE', \
             'DECEASED_FLAG', 'DISCIPLINARY_ACTION_FLAG', 'INEL_DUES_OR_RULES_FLAG', \
             'ADA_NOTES']

new_names = ['TADA_CODE', 'TADA_NAME', 'TADA_BAR_ADMISSION', 'TADA_DOB', 'TADA_RACE', \
             'TADA_SEX', 'TADA_PARTY', 'TADA_ALT_PARTY', 'TADA_LAW_SCHOOL', 'TADA_UNDERGRAD', \
             'TADA_UNVERIFIED_FLAG', 'TADA_INACTIVITY_DATE', 'TADA_DECEASED_FLAG', \
             'TADA_DISCIPLINARY_ACTION_FLAG', 'TADA_INEL_DUES_OR_RULES_FLAG', 'TADA_NOTES']

name_dict={}

for i in range(0,len(old_names)):
    name_dict[old_names[i]]=new_names[i]

data_merged = data_merged.rename(columns=name_dict)

312608 rows


In [145]:
#*********************************************************
#* bring information on defendant
#*********************************************************
#merge in dfdn_nodup on bofi_nbr
#Inner join

'''DELETED VARIABLES IN STATA CODE ARE NOT IN DATA_MERGED TABLE, COULD BE FROM 
   JUDGE CHARACTERISTICS TABLE?'''

'DELETED VARIABLES IN STATA CODE ARE NOT IN DATA_MERGED TABLE, COULD BE FROM \n   JUDGE CHARACTERISTICS TABLE?'

In [146]:
#data_merged = data_merged.drop(['DFDN_NAME', 'ADDR_1', 'ADDR_2', 'CITY', 'STATE', 'ZIP', \
#                                'CUSTODY_CODE', 'HOME_PHONE_NBR', 'BUSINESS_PHONE_NBR', \
#                                'DFDN_SSN', 'RESTRICTED_FLAG', 'JUVENILE_FLAG', \
#                                'CRIMINAL_FLAG', 'ALIAS_FLAG', 'DR_LIC_NBR', 'DR_LIC_STATE', \
#                                'DOB', 'DOB_PLACE', 'DFDN_HEIGHT_FEET', 'DFDN_HEIGHT_INCHES', \
#                                'DFDN_WEIGHT', 'DFDN_SEX', 'DFDN_RACE', 'DFDN_SKIN_COLOR', \
#                                'DFDN_EYE_COLOR', 'DFDN_HAIR_COLOR', 'FINGERPRINT_CLASS', \
#                                'DFDN_SMT', 'ADD_DATE', 'ADD_TIME', 'ADD_USER', 'CHANGE_DATE', \
#                                'CHANGE_TIME', 'CHANGE_USER', 'PHONIC_KEY', 'FBI_NBR', \
#                                'CCN_NBR', 'DFDN_DOB'], axis=1)
data_simple = pd.merge(data_merged, \
                 dfdn_nodup,\
                 on='BOFI_NBR', \
                 how='inner')
print('%s rows' %(data_simple.shape[0]))


312608 rows


In [147]:
data_simple.to_csv('data_simple.csv')